<a href="https://colab.research.google.com/github/qilin512/OUC-ComputerVision/blob/main/Texton/Textons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Textons



纹理图像是由重复的规则或随机pattern构成的一类特殊图像。纹理描述了物体的材质。纹理基元（texton）是描述纹理的一种有效方式，是纹理图像的bag-of-visual-words模型。本次作业实现2D纹理图像中的LM texton，是[原文](https://people.eecs.berkeley.edu/~malik/papers/LM-3dtexton.pdf)算法的简化。

## 数据集

我们使用[Brodatz数据集](https://sipi.usc.edu/database/database.php?volume=textures)的一部分来做实验。该部分数据和本笔记本一起发布，下载使用即可。这些图像都是单通道1024分辨率。在做分类和重构实验中，可以截取512或者256大小的子图像作为训练集和测试集。

## Texton

### LM filter bank

著名的牛津大学的VVG组提供了多种filter bank的[Matlab代码](https://www.robots.ox.ac.uk/~vgg/research/texclass/filters.html)。网上有人做了[Python的实现](https://github.com/tonyjo/LM_filter_bank_python)。这份代码有bug，输出结果和Matlab的不一致，我把这个仓库fork了一份，并修正了其中的bug，稍后会给原作者提PR。本次作业请使用我修改后的[Python代码](https://github.com/qilin512/LM_filter_bank_python/blob/master/lm.py)（我只修改了lm.py文件）。

输出的F矩阵（3维数组）包含了48个滤波器，每个都是49*49。

In [ ]:
# 使用我修改后的代码构造filter bank
# 可以把相关的代码拷贝过来，或者把该Python文件加入到工程中，作为jupyter笔记本调用的外部程序。

### 基于滤波器响应的纹理图像分类

使用filter bank对图像进行滤波，每个图像产生C个特征图（假设滤波器个数为C）。计算每个特征图的均值和方差，得到元素个数为2C的特征向量。使用KNN算法在特征向量的基础上对图像进行分类。

In [ ]:
# 使用特征图的简单统计特征（均值和方差）对图像进行分类

### 生成textons

使用LM filter bank对纹理图像进行滤波，并生成Texton。具体地，针对一幅纹理图像（比如$H\times W$大小），使用filter bank进行滤波（滤波器个数为C），得到$H \times W \times C$的特征图。每个像素对应一个C维的特征向量。针对这$H \times W$个特征向量进行K-means聚类，设置聚类类别数K=20。这K个聚类中心即称为该纹理图像的Texton。这种算法是稀疏编码在纹理图像上的应用，对应着bag-of-words模型里的字典。

In [ ]:
# 你的实现代码

### 基于texton的纹理分类

如果做纹理分类任务，将数据集中的所有纹理图像都进行上述处理，每个图像都产生K个texton。所有图像的texton放在一起，再按距离把相似的texton进行合并（这一步可选），最终得到Q个texton。这样，每个图像的每个像素属于哪个texton是可追溯的，能得到一幅或者一类图像在这Q个textons上的频率分布（直方图）。在推断的时候，给定一幅纹理图像，滤波后，每个像素对应的特征向量和Q个texton计算距离，每个像素都会对应一个texton，从而获得该图像在Q个texton上的分布直方图。将该直方图和训练集中的每类对应的直方图计算卡方距离（$\chi^2$ distance），即能进行分类。

In [ ]:
# 分类任务

### 基于texton的纹理重建

输入：一幅纹理图像、按照上述方法对这幅图像构建的texton字典。

目标：利用texton重构该图像。

算法：

1. 计算滤波器矩阵的伪逆。将F矩阵里的每个滤波器都拉成一个向量，C个滤波器构成一个新的F矩阵（$HW \times C$），求这个矩阵的伪逆$F^+$。
2. 每个texton都与$F^+$相乘，得到每个texton对应的重构patch。根据原图像的每个像素值对应的texton，找到对应的重构patch，将该patch的中心元素作为重构图像的该像素的像素值。

In [ ]:
# 重构任务的代码

## 可学习的特征

filter bank设计得很巧妙，是学者的脑力贡献，如何运用filter bank提取的丰富特征？我们前面用过统计特征（均值和方差）和texton（聚类pooling）。如果能从数据中学习到参数，岂不是更好？最后让我们借鉴深度学习里的CNN（convolutional neural network）。

请设计一个两层的CNN网络，第一个卷积层设置为我们的filter bank：48个卷积核（滤波器），大小为$49 \times 49$（够大核的）。第二层卷积层的卷积核个数为类别数，大小为$1 \times 1$。网络的第三层为average pooling层。最后接softmax输出类别概率。

第一层卷积层因为使用设计好的filter bank，所以需要加载现成的参数，相当于预训练的部分模型参数。

实验分两部分。第一部分实验freeze第一个卷积层，只学习第二个卷积层。第二部分实验学习网络的全部参数，相当于对filter bank进行微调fine tuning。对比一下两个部分的分类效果，并把第二部分的微调后的第一个卷积层的滤波器保存并可视化出来，看看和学者们设计的滤波器相比有什么变化。

In [ ]:
# 华为智能基座吗？
# import mindspore as torch

如果微调后的滤波器分类效果更好的话，可否用在重构任务上？重构的效果会不会更好？如果重构效果更好的话，可否用深度网络实现重构任务？